# C-Eval

A Multi-Level Multi-Discipline Chinese Evaluation Suite for Foundation Models

Dataset: <https://huggingface.co/datasets/ceval/ceval-exam>

ceval 的数据分为 test/val/dev 三类，其中 test/val 没有标准答案，而  dev 有标准答案。
这里有两类任务的示例：

1. 使用标准答案进行匹配：`ceval-with-answer` 任务，使用 `ceval` 数据集的 `dev` 部分。
2. 使用 GPT-4 进行判卷：`ceval-llm-judge` 任务，使用 `ceval` 数据集的 `test/val` 部分。

## 1. 数据处理

ceval 的数据格式为 huggingface dataset，需要转换为本工具依赖的 jsonl 格式。
同时仅作为示例的话，每个任务仅保留 5 条数据。

In [ ]:
%pip install langeval-cli
%pip install datasets

In [15]:
import json
from datasets import load_dataset
print(">>> ceval-with-answer.jsonl")
dev_dataset = load_dataset(r"ceval/ceval-exam", name="computer_network", split="dev", streaming=True)
with open("ceval-with-answer.jsonl", 'w') as f:
    for data in dev_dataset.take(5):
        print(data)
        f.write(json.dumps(data, ensure_ascii=False) + '\n')

print(">>> ceval-llm-judge.jsonl")
test_dataset = load_dataset(r"ceval/ceval-exam", name="computer_network", split="test", streaming=True)
with open("ceval-llm-judge.jsonl", 'w') as f:
    for data in test_dataset.take(5):
        print(data)
        f.write(json.dumps(data, ensure_ascii=False) + '\n')

>>> ceval-with-answer.jsonl
{'id': 0, 'question': '下列设备属于资源子网的是____。', 'A': '计算机软件', 'B': '网桥', 'C': '交换机', 'D': '路由器', 'answer': 'A', 'explanation': '1. 首先，资源子网是指提供共享资源的网络，如打印机、文件服务器等。\r\n2. 其次，我们需要了解选项中设备的功能。网桥、交换机和路由器的主要功能是实现不同网络之间的通信和数据传输，是通信子网设备。而计算机软件可以提供共享资源的功能。'}
{'id': 1, 'question': '滑动窗口的作用是____。', 'A': '流量控制', 'B': '拥塞控制', 'C': '路由控制', 'D': '差错控制', 'answer': 'A', 'explanation': '1. 滑动窗口是一种流量控制机制，用于控制发送方向接收方发送数据的速率，以避免接收方无法处理过多的数据而导致数据丢失或拥塞。'}
{'id': 2, 'question': '在OSI参考模型中，直接为会话层提供服务的是____。', 'A': '应用层', 'B': '表示层', 'C': '传输层', 'D': '网络层', 'answer': 'C', 'explanation': '1. 直接为会话层提供服务的是会话层的下一层，即传输层。'}
{'id': 3, 'question': '协议是指在____之间进行通信的规则或约定。', 'A': '同一结点的上下层', 'B': '不同结点', 'C': '相邻实体', 'D': '不同结点对等实体', 'answer': 'D', 'explanation': '1. 协议是指在不同结点对等实体之间进行通信的规则或约定。'}
{'id': 4, 'question': '主机甲与主机乙之间使用后退N帧协议(GBN)传输数据，甲的发送窗口尺寸为1000，数据帧长为1000字节，信道带宽为100Mbps，乙每收到一个数据帧立即利用一个短帧(忽略其传输延迟)进行确认，若甲、乙之间的单向传播延迟是50ms，则甲可以达到的最大平均数据传输速率约为____。', 'A': '10Mbps', 'B': '20Mbps', 'C': '80Mbp

## 2. `ceval-with-answer` 任务，使用标准答案进行匹配。

```bash
# 可以先使用 sample 参数来测试单条数据
export OPENAI_API_KEY="sk-xxxxx"
langeval -vv run ceval-with-answer.yaml --sample

# 全量测试
langeval run ceval-with-answer.yaml
```

输出示例：
```bash
>>> Loaded task from ../examples/ceval/ceval-with-answer.yaml successfully, task_id: 2311021343-7dd9
Output dir: output/2311021343-7dd9
Output dir created: output/2311021343-7dd9
>>> Start running task 2311021343-7dd9
[runner.start] start task runner
>>> Task 2311021343-7dd9 running...
[runner._run] start task run : parallelism=1 timeout=30 rounds=2
[runner._run] task run: 5 * 2 = 10
[runner._run] task sample to 1 data.
[runner._run] task progress 1/0/1, result: inputs={'id': 0, 'question': '下列设备属于资源子网的是____。', 'A': '计算机软件', 'B': '网桥', 'C': '交换机', 'D': '路由器', 'answer': 'A', 'explanation': '1. 首先，资源子网是指提供共享资源的网络，如打印机、文件服务器等。\r\n2. 其次，我们需要了解选项中设备的功能。网桥、交换机和路由器的主要功能是实现不同网络之间的通信和数据传输，是通信子网设备。而计算机软件可以提供共享资源的功能。'} run_error='' run_outputs={'reasoning': '资源子网是指由一组计算机和相关设备组成的网络，用于共享资源和信息。计算机软件不属于设备，因此排除选项 A。网桥是用于连接两个局域网的设备，不属于资源子网的设备，因此排除选项 B。交换机是用于连接多个计算机的设备，属于资源子网的设备，因此选项 C 是正确答案。路由器是用于连接不同网络的设备，不属于资源子网的设备，因此排除选项 D。', 'choice': 'C', '_text': '\n\n```json\n{\n    "reasoning": "资源子网是指由一组计算机和相关设备组成的网络，用于共享资源和信息。计算机软件不属于设备，因此排除选项 A。网桥是用于连接两个局域网的设备，不属于资源子网的设备，因此排除选项 B。交换机是用于连接多个计算机的设备，属于资源子网的设备，因此选项 C 是正确答案。路由器是用于连接不同网络的设备，不属于资源子网的设备，因此排除选项 D。",\n    "choice": "C"\n}\n```'} run_elapsed_secs=3.299008 eval_error='' eval_outputs={'score': 0} eval_elapsed_secs=0.037781
[runner._run] task finished
>>> Task 2311021343-7dd9 finish: TaskRunnerStatus.FINISHED.
────────────────────────────────────────────────────────────────────────────────────────────────── >>> Task Info ──────────────────────────────────────────────────────────────────────────────────────────────────
ID: 2311021343-7dd9
Status: TaskRunnerStatus.FINISHED
Progress: 1/0/1
Output JSONL: output/2311021343-7dd9/output.jsonl
───────────────────────────────────────────────────────────────────── Result Sample, see output/2311021343-7dd9/output.jsonl for all results. 
```

## 3. `ceval-llm-judge` 任务，使用 GPT-4 进行判卷。

```bash
export OPENAI_API_KEY="sk-xxxxx"
# 单个问题
langeval run ceval-llm-judge.yaml --sample

# 全量测试
langeval run ceval-llm-judge.yaml
```